In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import sys, os

In [ ]:
automation_dir = "/mnt/analysis/e17023/Adam/GADGET2/"

In [15]:
P0 = 'p'
P1 = 'a'
N = 100
N_full = 20000
uniform_params = {
    'E0' : [500, 2200],
    'E1' : [0.001, 0.01],
    'CD' : [1, 3],
    'CDH' : [1, 3]}
pv = 1.3
multiplicative_params = {
    'Threshold' : [100,4],
    'EIonize' : [22.3, pv],
    'Fano' : [0.24, pv],
    'CoefL' : [0.000114, pv],
    'CoefT' : [0.00284, pv],
    'Gain' : [8000, pv],
    'GETGain' : [120, pv],
    'PeakingTime' : [1014, pv],
    'GasPressure' : [800, pv]}

In [16]:
param_df = pd.DataFrame(columns=['Sim', 'Status', 'P0', 'E0', 'P1', 'E1', 'N'])
N_sim = N_full // N
for i in range(N_sim):
    param_df = param_df.append({'Sim' : f'VarSimCDA{i}', 'Status' : 0, 'N' : N}, ignore_index=True)

param_df['P0'] = P0
param_df['P1'] = P1

for key, value in uniform_params.items():
    param_df[key] = np.random.randint(value[0], value[1]+1, N_sim)
for key, value in multiplicative_params.items():
    dev_k = np.random.uniform(-1,1,N_sim)
    param_df[key] = value[0] * (value[1] ** dev_k)

# integer keys fix
for key in ['CD', 'CDH', 'Threshold', 'Gain', 'GETGain', 'PeakingTime', 'E0', 'E1', 'N']:
    param_df[key] = param_df[key].astype(int)

# if energy is 0, set to 0.001 to avoid zero-energy errors
param_df.loc[param_df['E0'] == 0, 'E0'] = 0.001
param_df.loc[param_df['E1'] == 0, 'E1'] = 0.001

In [17]:
try: 
    old_params = pd.read_csv(automation_dir+'.input/parameters_old.csv')
except:
    old_params = pd.read_csv(automation_dir+'parameters.csv')
old_params.to_csv(automation_dir+'.input/parameters_old.csv', index=False)
new_params = old_params.append(param_df, ignore_index=True)

# set nan values in columns to central values
for key, value in uniform_params.items():
    new_params.loc[new_params[key].isna(), key] = np.mean(value)
for key, value in multiplicative_params.items():
    new_params.loc[new_params[key].isna(), key] = value[0]
new_params

new_params.to_csv(automation_dir+'parameters.csv', index=False)